In [1]:
import requests as req
from bs4 import BeautifulSoup
from IPython.display import display
import pandas as pd
import json

In [ ]:
# 參考網站: https://mis.twse.com.tw/stock/index.jsp (基本市況報導)

In [78]:
def stock_list(stock_number_list):
    
    s_list =[]
    for stock_num in stock_number_list:
        url = "https://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_" + stock_num + ".tw&json=1&delay=0&_=1581253418747"
        r = req.get(url)
        info  = json.loads(r.text)
        info = info["msgArray"][0]
    
        info_index = ['c','n','z','tv','v','o','h','l','y']
        column = ['股票代號','公司簡稱','當盤成交價','當盤成交量','累積成交量','開盤價','最高價','最低價','昨收價']
    
        stock_info ={}
        for i in range(len(info_index)):
            stock_info[column[i]] = info[info_index[i]]
    
        s_list.append(stock_info)
    
    return s_list

In [79]:
stock_number_list = [i for i in input().split(" ")]
stockinfo_list = stock_list(stock_number_list)

#stockinfo_list

5007 8046 3338 2303 3017 3037 3231 6153 2449 2317


In [80]:
#stock形式為list

def stock_table(stock):
    
    if len(stock) > 1:
        first_table = pd.DataFrame(stock[0], index = ["股票資訊"])
        
        for stock in stock[1:]:
            
            stock_info = pd.DataFrame(stock, index = ["股票資訊"])
            stock_table  = pd.concat([first_table,stock_info], axis = 0)
            first_table = stock_table
        
        stock_info_table = stock_table
        
    else:
        stock_info_table = pd.DataFrame(stock[0], index = ["股票資訊"])
     

    stock_info_table = stock_info_table.astype({'當盤成交價': "float", '當盤成交量':"int64", '累積成交量':"int64", '開盤價':"float", '最高價':"float", '最低價':"float",'昨收價':"float"})
    
    stock_info_table["漲跌幅"] = stock_info_table.loc["股票資訊","當盤成交價"] - stock_info_table.loc["股票資訊","昨收價"]
    #stock_info_table.loc[:,["漲跌幅"]] = stock_info_table.loc[:,["漲跌幅"]].astype(float)
    
    
    return stock_info_table

In [81]:
stock_table(stockinfo_list)

,股票代號,公司簡稱,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌幅
股票資訊,5007,三星,50.20,5,27,50.10,50.20,50.00,50.20,0.00
股票資訊,8046,南電,47.00,426,8861,44.80,47.40,44.80,45.05,1.95
股票資訊,3338,泰碩,75.60,228,2927,75.20,76.60,75.20,75.60,0.00
股票資訊,2303,聯電,16.90,4701,79377,16.65,16.95,16.60,16.70,0.20
股票資訊,3017,奇鋐,33.80,257,6614,33.55,34.20,33.10,33.55,0.25
股票資訊,3037,欣興,42.85,2860,61447,40.60,42.90,40.35,40.30,2.55
股票資訊,3231,緯創,28.25,657,7473,28.00,28.30,27.95,28.00,0.25
股票資訊,6153,嘉聯益,32.40,431,6246,32.05,32.90,32.05,32.00,0.40
股票資訊,2449,京元電子,36.00,1256,12444,35.60,36.10,35.20,35.60,0.40
股票資訊,2317,鴻海,85.50,6270,55982,83.00,85.60,83.00,82.90,2.60


In [82]:
s1 = stock_table(stockinfo_list).copy()

s1.insert(0,"公司",s1["股票代號"] + s1["公司簡稱"])
s1 = s1.drop(["股票代號","公司簡稱"],axis =1)

s1 = s1.set_index("公司")
s1

,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌幅
公司,,,,,,,,
5007三星,50.20,5,27,50.10,50.20,50.00,50.20,0.00
8046南電,47.00,426,8861,44.80,47.40,44.80,45.05,1.95
3338泰碩,75.60,228,2927,75.20,76.60,75.20,75.60,0.00
2303聯電,16.90,4701,79377,16.65,16.95,16.60,16.70,0.20
3017奇鋐,33.80,257,6614,33.55,34.20,33.10,33.55,0.25
3037欣興,42.85,2860,61447,40.60,42.90,40.35,40.30,2.55
3231緯創,28.25,657,7473,28.00,28.30,27.95,28.00,0.25
6153嘉聯益,32.40,431,6246,32.05,32.90,32.05,32.00,0.40
2449京元電子,36.00,1256,12444,35.60,36.10,35.20,35.60,0.40


# 新增顏色標記(漲 or 跌)

In [28]:
def tablecolor(value):
    if value > 0:
        color = "red"
    elif value < 0:
        color = "green"
    else:
        color = "blake"
    return "color: {}".format(color)

In [84]:
s1.style.applymap(tablecolor, subset = ["漲跌幅"])

,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌幅
公司,,,,,,,,
5007三星,50.2,5,27,50.1,50.2,50,50.2,0
8046南電,47,426,8861,44.8,47.4,44.8,45.05,1.95
3338泰碩,75.6,228,2927,75.2,76.6,75.2,75.6,0
2303聯電,16.9,4701,79377,16.65,16.95,16.6,16.7,0.2
3017奇鋐,33.8,257,6614,33.55,34.2,33.1,33.55,0.25
3037欣興,42.85,2860,61447,40.6,42.9,40.35,40.3,2.55
3231緯創,28.25,657,7473,28,28.3,27.95,28,0.25
6153嘉聯益,32.4,431,6246,32.05,32.9,32.05,32,0.4
2449京元電子,36,1256,12444,35.6,36.1,35.2,35.6,0.4


# 在已建立的dataframe內 用股票代號 or 公司簡稱做查詢 

In [89]:
name = input("stock number or name would like to search: ")

def search(name):
    for i in s1.index:
        if name in i:
            return s1.loc[[i],:]
        
search(name)

stock number or name would like to search: 2449


,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌幅
公司,,,,,,,,
2449京元電子,36.0,1256,12444,35.6,36.1,35.2,35.6,0.4


# 查汽車工業所有股票為例

In [2]:
cars = pd.read_excel("stockinfo.xlsx", sheet_name = "汽車", header = 0)

car_list = []
for i in cars["代號"]:
    car_list.append(i)
    
car_lists = [str(i) for i in car_list]
print(car_lists)

['1338', '1339', '1512', '1521', '1522', '1524', '1525', '1533', '1536', '1568', '1587', '1592', '2115', '2201', '2204', '2206', '2207', '2227', '2228', '2231', '2233', '2236', '2239', '2243', '2497', '3346', '4551', '4557', '6288', '6605', '1319', '1338', '1339', '1512', '1521', '1522', '1524', '1525', '1533', '1536', '1568', '1587', '1592', '2115', '2201', '2204', '2206', '2207', '2227', '2228', '2231', '2233', '2236', '2239', '2243', '2497', '3346', '4551', '4557', '6288', '6605']


In [109]:
car = stock_list(car_lists)
car_stock_info = stock_table(car)
car_stock_info = car_stock_info.set_index("股票代號")  #重新設定index
car_stock_info = car_stock_info.sort_values("當盤成交量", ascending = False) 

car_stock_info

,公司簡稱,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌幅
股票代號,,,,,,,,,
2201,裕隆,20.05,253,9010,19.65,20.10,19.65,19.65,0.4
2201,裕隆,20.05,253,9010,19.65,20.10,19.65,19.65,0.4
6288,聯嘉,25.55,219,4234,25.70,25.95,25.50,25.45,0.1
6288,聯嘉,25.55,219,4234,25.70,25.95,25.50,25.45,0.1
2207,和泰車,630.00,103,338,625.00,640.00,623.00,624.00,6.0
...,...,...,...,...,...,...,...,...,...
2115,六暉-KY,38.00,1,37,38.00,38.00,37.50,37.50,0.5
2236,百達-KY,19.80,1,20,19.85,19.90,19.80,19.80,0.0
2239,英利-KY,83.00,1,123,80.50,83.30,80.50,80.10,2.9
